In [12]:
import math, os, sys
import json
import plotly.express as px
import pandas as pd
import numpy as np
import cv2
from skimage import io
from matplotlib import pyplot as plt
from collections import defaultdict
import random, string
from scipy.interpolate import UnivariateSpline, SmoothBivariateSpline,splprep, splev
from datetime import datetime
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/activebrainatlas')
sys.path.append(PATH)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()
from timeit import default_timer as timer

In [ ]:
def random_string():
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=40))

In [ ]:
from plotly.offline import plot
import plotly.graph_objects as go

In [2]:
from neuroglancer.models import UrlModel, LayerData, Structure, Transformation
from brain.models import Animal, ScanRun
from neuroglancer.atlas import brain_to_atlas_transform, align_atlas

In [14]:
brains = list(LayerData.objects.filter(active=True)\
.filter(input_type__input_type__in=['manual'])\
.filter(layer='COM')\
.filter(active=True)\
.exclude(prep_id='Atlas')\
.values_list('prep_id', flat=True).distinct().order_by('prep_id'))
print(animals)
title = 'Rigid Alignment Error for ' + ", ".join(brains)
print(title)

['DK39', 'DK41', 'DK43', 'DK46', 'DK52', 'DK54', 'DK55', 'MD589']
Rigid Alignment Error for DK39, DK41, DK43, DK46, DK52, DK54, DK55, MD589


In [29]:
def get_centers_dict(prep_id, input_type_id=0, person_id=None):
    start = timer()
    rows = LayerData.objects.filter(prep__prep_id=prep_id)\
        .filter(active=True).filter(layer='COM')\
            .order_by('structure', 'updated')
    if input_type_id > 0:
        rows = rows.filter(input_type_id=input_type_id)
    if person_id is not None:
        rows = rows.filter(person_id=person_id)
        
        
    structure_dict = {}
    structures = Structure.objects.filter(active=True).all()
    for structure in structures:
        structure_dict[structure.id] = structure.abbreviation
    row_dict = {}
    for row in rows:
        structure_id = row.structure_id
        abbreviation = structure_dict[structure_id]
        row_dict[abbreviation] = [row.x, row.y, row.section]

    end = timer()
    print(f'get centers_dict took {end - start} seconds')
    return row_dict

def get_common_structure(brains):
    start = timer()
    common_structures = set()
    for brain in brains:
        common_structures = common_structures | set(get_centers_dict(brain).keys())
    common_structures = list(sorted(common_structures))
    end = timer()
    print(f'get common structures took {end - start} seconds')
    return common_structures

In [30]:
%%time
get_common_structure(brains)

get centers_dict took 0.0118839459028095 seconds
get centers_dict took 0.009294650983065367 seconds
get centers_dict took 0.007419033092446625 seconds
get centers_dict took 0.006368004018440843 seconds
get centers_dict took 0.007563368999399245 seconds
get centers_dict took 0.008908972959034145 seconds
get centers_dict took 0.006186263985000551 seconds
get centers_dict took 0.005829861969687045 seconds
get common structures took 0.06567802792415023 seconds
CPU times: user 47.7 ms, sys: 204 µs, total: 47.9 ms
Wall time: 65.7 ms


['10N_L',
 '10N_R',
 '12N',
 '3N_L',
 '3N_R',
 '4N_L',
 '4N_R',
 '5N_L',
 '5N_R',
 '6N_L',
 '6N_R',
 '7N_L',
 '7N_R',
 '7n_L',
 '7n_R',
 'AP',
 'Amb_L',
 'Amb_R',
 'DC_L',
 'DC_R',
 'IC',
 'LC_L',
 'LC_R',
 'LRt_L',
 'LRt_R',
 'PBG_L',
 'PBG_R',
 'Pn_L',
 'Pn_R',
 'RtTg',
 'SC',
 'SNC_L',
 'SNC_R',
 'SNR_L',
 'SNR_R',
 'Sp5C_L',
 'Sp5C_R',
 'Sp5I_L',
 'Sp5I_R',
 'Tz_L',
 'Tz_R',
 'VCA_L',
 'VCA_R',
 'VCP_L',
 'VCP_R',
 'VLL_L',
 'VLL_R']

In [ ]:
users = User.objects.filter(id__in=com_users)

In [ ]:
Animal.objects.filter(layerdata__active=True).distinct().order_by()

In [ ]:
count = LayerData.objects.filter(prep_id='Atlas').filter(active=True).count()        
print(count, type(count))

In [ ]:
id = 283 # has points
#id = 307 # not points
urlModel = UrlModel.objects.get(pk=id)
urlModels = [urlModel]

In [ ]:
urlModels = UrlModel.objects.filter(vetted=True).all()
print(len(urlModels))

In [ ]:
dfs = []
for urlModel in urlModels:
    json_txt = urlModel.url
    layers = json_txt['layers']
    for layer in layers:
        if 'annotations' in layer:
            name = layer['name']
            annotation = layer['annotations']
            d = [row['point'] for row in annotation if 'point' in row and 'pointA' not in row]
            df = pd.DataFrame(d, columns=['X', 'Y', 'Section'])
            df['Section'] = df['Section'].astype(int)
            df['Layer'] = name
            structures = [row['description'] for row in annotation if 'description' in row]
            if len(structures) != len(df):
                structures = ['' for row in annotation if 'point' in row and 'pointA' not in row]
            df['Description'] = structures
            df = df[['Layer', 'Description', 'X', 'Y', 'Section']]
            dfs.append(df)
    if len(dfs) == 0:
        result = None
    elif len(dfs) == 1:
        result = dfs[0]
    else:
        result = pd.concat(dfs)

In [ ]:
result.head()

In [ ]:
json_txt = urlModel.url
layer_list = []
layers = json_txt['layers']
for layer in layers:
    if 'annotations' in layer:
        layer_name = layer['name']
        layer_list.append(layer_name)
print(layer_list)
print(urlModel.layers)

In [ ]:
from django.db.models import Count
result = Task.objects.all().filter(lookup__channel__in=[0,1]).filter(lookup__downsample=False)\
.filter(prep__active=True)\
.values('prep_id').annotate(total=Count('prep_id')).order_by('prep_id')

In [ ]:
animals = Animal.objects.filter(active=True).order_by('prep_id').all()
tasks = TaskView.objects.filter(prep_id__in=animals).order_by('prep_id').all()
lookups = ProgressLookup.objects.filter(channel__in=[0,1]).filter(downsample=False).order_by('id').all()

al = []
x = [] 
i = 0
for animal in animals:
    if animal.aliases_1 is None:
        continue
    al.append(str(animal.prep_id + '-' + animal.aliases_1))
    x.append(result[i]['total'])
    i += 1

lookup_list = []
id_list = []
for i, lookup in enumerate(lookups):
    id_list.append(i)
    lookup_list.append(lookup.description)

In [ ]:
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0x00FF00, 0x00FFFF), range(n)))
colors = get_colors(len(x))
#colors = [x*19 for x in range(len(x))]
marker={'color': colors, 'colorscale': 'Viridis'}
fig = go.Figure(data=[go.Bar(x=x, y=al, orientation='h', marker=marker )])
fig.update_layout(xaxis_showgrid=True, yaxis_showgrid=True, height=600)
fig.update_xaxes(ticks='outside',tickwidth=2, tickangle=45, tickcolor='crimson', ticklen=10, range=[0,limit-1])
fig.update_layout(autosize=True, xaxis=dict(tickmode='array', tickvals=id_list, ticktext=lookup_list),
                  margin=dict(l=20, r=20, t=20, b=280),
                  paper_bgcolor="LightSteelBlue",
                  )
fig.show()

In [ ]:
df = pd.DataFrame(list(LayerData.objects.all().values()))
df2 = df.groupby([df['created'].dt.date])[["x", "y", "section"]].mean()
df2.head(2)

In [ ]:
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH3/thumbnail_aligned'
section = 201
file = f'{section}.tif' 
infile = os.path.join(INPUT, file)
prep_id = 'DK52'
layer_name = 'PremotorShape'
input_type_id = 5
rows = LayerData.objects.filter(prep_id=prep_id).filter(layer=layer_name)\
    .filter(input_type_id=input_type_id).filter(active=True).order_by('section','id').all()
print(len(rows))

In [ ]:
data_dict = defaultdict(list)
for row in rows:
    id = row.segment_id
    x = int(row.x // 32)
    y = int(row.y // 32)
    section = row.section
    data_dict[(id,section)].append((x,y))

In [ ]:
for (id,section), points in data_dict.items():
    if section == 201:
        print(id,section, len(points))

In [ ]:
data = []
for (id,section), points in data_dict.items():
    lp = len(points)
    for i in  range(lp):
        tmp_dict = {}
        pointA = points[i]
        try:
            pointB = points[i+1]
        except IndexError as error:
            pointB = points[0]
        tmp_dict['id'] = random_string()
        tmp_dict['pointA'] = [pointA[0], pointA[1], section]
        tmp_dict['pointB'] = [pointB[0], pointB[1], section]
        tmp_dict['type'] = 'line'
        tmp_dict['description'] = ""
        data.append(tmp_dict)

In [ ]:
id = 64689
section = 201
points = data_dict[(id,section)]
print(len(points), id, section)
#for (k,v), points in data_dict.items():
#    print(k,v, len(points)
print(points)


In [ ]:
def interpolate_list(points, new_len):
    x = [v[0] for v in points]
    y = [v[1] for v in points]
    vx = np.array(x)
    vy = np.array(y)
    indices = np.arange(0,len(points))
    new_indices = np.linspace(0,len(points)-1,new_len)
    splx = UnivariateSpline(indices,vx,k=3,s=0)
    x_array = splx(new_indices)
    sply = UnivariateSpline(indices,vy,k=3,s=1)
    y_array = sply(new_indices)
    arr_2d = np.concatenate([x_array[:,None],y_array[:,None]], axis=1)
    a = list(map(tuple, arr_2d))
    return a

def interpolate(points, new_len):
    pu = points.astype(int)
    indexes = np.unique(pu, axis=0, return_index=True)[1]
    points = np.array([pu[index] for index in sorted(indexes)])
    addme = points[0].reshape(1,2)
    points = np.concatenate((points,addme), axis=0)

    tck, u = splprep(points.T, u=None, s=3, per=1) 
    u_new = np.linspace(u.min(), u.max(), new_len)
    x_array, y_array = splev(u_new, tck, der=0)
    arr_2d = np.concatenate([x_array[:,None],y_array[:,None]], axis=1)
    return arr_2d



In [ ]:
points

In [ ]:
points = sub_dict
new_len = max(len(points),20)
delta = (len(points)-1) / (new_len-1)

#outp = [interpolate(points, i*delta) for i in range(new_len)]
outp = interpolate(points, new_len)

outp = [(int(i), int(j)) for i,j in outp]

print('points',(points))
print('lin out',outp)
print('len original array', len(points))
print('len new array', len(outp))




In [ ]:
# python lists of x,y dst control points and zx=xsrc-xdiff, zy=ysrc-ydiff offsets to be interpolated over full image of size 129x129
#x = [8.5, 20.5, 33.5, 48.5, 64.5, 80.5, 95.5, 109.5, 121.5, 5.5, 17.5, 31.5, 46.5, 64.5, 81.5, 97.5, 111.5, 123.5, 2.5, 14.5, 29.5, 45.5, 64.5, 83.5, 99.5, 113.5, 125.5, 1.5, 12.5, 26.5, 43.5, 64.5, 85.5, 103.5, 116.5, 127.5, 0.5, 11.5, 24.5, 41.5, 64.5, 87.5, 103.5, 117.5, 128.5, 1.5, 12.5, 25.5, 42.5, 64.5, 86.5, 103.5, 116.5, 127.5, 2.5, 14.5, 28.5, 45.5, 64.5, 83.5, 100.5, 114.5, 125.5, 5.5, 17.5, 30.5, 46.5, 64.5, 81.5, 97.5, 111.5, 123.5, 8.5, 19.5, 33.5, 48.5, 64.5, 80.5, 95.5, 109.5, 121.5]
#y = [7.5, 5.5, 3.5, 1.5, 1.5, 1.5, 3.5, 5.5, 7.5, 20.5, 16.5, 14.5, 12.5, 11.5, 12.5, 15.5, 16.5, 19.5, 33.5, 31.5, 28.5, 26.5, 24.5, 26.5, 28.5, 31.5, 33.5, 48.5, 47.5, 45.5, 42.5, 40.5, 42.5, 45.5, 46.5, 48.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 80.5, 81.5, 83.5, 86.5, 87.5, 86.5, 83.5, 81.5, 80.5, 95.5, 97.5, 100.5, 103.5, 104.5, 102.5, 100.5, 97.5, 95.5, 109.5, 111.5, 114.5, 116.5, 117.5, 116.5, 114.5, 111.5, 109.5, 121.5, 123.5, 125.5, 127.5, 127.5, 127.5, 125.5, 123.5, 120.5]
#zx = [119.5, 123.5, 126.5, 127.5, 127.5, 127.5, 128.5, 130.5, 134.5, 122.5, 126.5, 128.5, 129.5, 127.5, 126.5, 126.5, 128.5, 132.5, 125.5, 129.5, 130.5, 130.5, 127.5, 124.5, 124.5, 126.5, 130.5, 126.5, 131.5, 133.5, 132.5, 127.5, 122.5, 120.5, 123.5, 128.5, 127.5, 132.5, 135.5, 134.5, 127.5, 120.5, 120.5, 122.5, 127.5, 126.5, 131.5, 134.5, 133.5, 127.5, 121.5, 120.5, 123.5, 128.5, 125.5, 129.5, 131.5, 130.5, 127.5, 124.5, 123.5, 125.5, 130.5, 122.5, 126.5, 129.5, 129.5, 127.5, 126.5, 126.5, 128.5, 132.5, 119.5, 124.5, 126.5, 127.5, 127.5, 127.5, 128.5, 130.5, 134.5]
#zy = [120.5, 122.5, 124.5, 126.5, 126.5, 126.5, 124.5, 122.5, 120.5, 123.5, 127.5, 129.5, 131.5, 132.5, 131.5, 128.5, 127.5, 124.5, 126.5, 128.5, 131.5, 133.5, 135.5, 133.5, 131.5, 128.5, 126.5, 127.5, 128.5, 130.5, 133.5, 135.5, 133.5, 130.5, 129.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 127.5, 126.5, 124.5, 121.5, 120.5, 121.5, 124.5, 126.5, 127.5, 128.5, 126.5, 123.5, 120.5, 119.5, 121.5, 123.5, 126.5, 128.5, 130.5, 128.5, 125.5, 123.5, 122.5, 123.5, 125.5, 128.5, 130.5, 134.5, 132.5, 130.5, 128.5, 128.5, 128.5, 130.5, 132.5, 135.5]

x = [v[0] for v in points]
y = [v[1] for v in points]
zx = [200 for x in points]
# convert python lists to numpy arrays
ax = np.asarray(x)
ay = np.asarray(y)
azx = np.asarray(zx)
azy = np.asarray(zy)
# define bbox of interpolated data
minx = min(x)
maxx = max(x)
miny = min(y)
maxy = max(y)
bbox=[minx, maxx, miny, maxy]
# convert bbox to numpy array
abbox = np.asarray(bbox)
# do interpolations
xd = SmoothBivariateSpline(ax, ay, azx, w=None, bbox=abbox, kx=1, ky=1)
interp_func = SmoothBivariateSpline(ax, ay, azx, s=0.1)

In [ ]:
pts = "1402.4326171875,578.2587890625,1402.4322981366458,586.6683229813661,1391.919875776397,576.1559006211173"

In [ ]:
pts = np.array([tuple(map(float, x.split())) for x in pts.strip().split(',')])

In [ ]:
vertices = pts.reshape(pts.shape[0]//2, 2).astype(np.float64)

In [ ]:
addme = vertices[0].reshape(1,2)
vertices = np.concatenate((vertices,addme), axis=0)
vertices

In [ ]:
new_len = 100
tck, u = splprep(vertices.T, u=None, s=0.0, per=0) 
u_new = np.linspace(u.min(), u.max(), new_len)
x_array, y_array = splev(u_new, tck, der=0)

plt.plot(vertices[:,0], vertices[:,1], 'ro')
plt.plot(x_array, y_array, 'b--')
plt.show()

In [ ]:
#Set the resolution:
x_res=20
#y_res=int( ( (maxy-miny) / (maxx-minx) )*x_res )
y_res=x_res
#Make a grid
grid_x, grid_y = np.mgrid[minx:maxx:x_res*1j, miny:maxy:y_res*1j]
#xd.ev(grid_x[0,:], grid_y[0,:])

In [ ]:
test_x = np.arange(minx, maxx, 0.5)
test_y = np.arange(miny, maxy, 0.5)
grid_x, grid_y = np.meshgrid(test_x, test_y)

In [ ]:
interp_func(test_x, test_y)

In [ ]:
xd(test_x, test_y)

In [ ]:
file = f'{section}.tif' 
infile = os.path.join(INPUT, file)

img = io.imread(infile, img_num=0)
color = (255,255,255)
for (id,s), points in data_dict.items():
        if s == section and len(points) > 3:
            new_len = max(len(points),5000)
            print(new_len)
            outp = interpolate(points, new_len)
            points = [(int(i), int(j)) for i,j in outp]

            lp = len(points)
            for i in  range(new_len):
                pointA = points[i]
                try:
                    pointB = points[i+1]
                except IndexError as error:
                    pointB = points[0]
                cv2.line(img, pointA, pointB, color, 2)

In [ ]:
#cv2.polylines(img, [points], isClosed=True, color=(0,0,0), thickness=5)
fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('{} section:{}'.format(prep_id, section))
plt.show()

In [ ]:
ids = {'infrahypoglossal': 200, 'perifacial': 210, 'suprahypoglossal': 220}

In [ ]:
[str(number) for number, label in ids.items()]

In [ ]:
[str(label) for number, label in ids.items()]

In [ ]:
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH3/thumbnail_aligned'
files = sorted(os.listdir(INPUT))
for i,file in enumerate(files):
    print(i,file)